In [1]:
from keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Embedding,  BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Model,Sequential
from keras.utils import to_categorical
from keras import regularizers
from keras.optimizers import SGD
from clr_callback import *
%matplotlib inline
import time
import pickle
import os
import math
import torch.nn as nn
from torch.nn.modules.utils import _triple
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import EarlyStopping
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report



In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
SPLIT_METHOD_child = "CHILD"
SPLIT_METHOD_session = "SESSION"
SPLIT_METHOD_random = "RANDOM"

# WITH FACE, BODY, HANDS (411) =>
FEAT_NUM_all = 201

# WITH BODY AND HANDS (201) =>
FEAT_NUM_body = (25 + 21 + 21) * 3

# WITH FACE ONLY (210) =>
FEAT_NUM_face = 70 * 3

SEQ_LENGTH = 1

NB_CLASS = 6
CSV_FILE = "ASD_36_res1.csv"
VARS_DIR = "var"


In [4]:
import pandas
data = pandas.read_csv('ASD_36_res1.csv')

In [5]:
X = data.iloc[:, 3:78]
y = data['engagement']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=1)

In [6]:
X_train.shape


(253124, 75)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [8]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(227811, 75)
(25313, 75)
(41207, 75)


In [9]:
X_train = X_train.to_numpy()
X_test =X_test.to_numpy()
X_val = X_val.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_val = y_val.to_numpy()

In [10]:
type(X_train)

numpy.ndarray

In [11]:

early_stop = EarlyStopping(monitor='loss', patience=2)


In [12]:
xx = tf.convert_to_tensor(X_train.reshape(227811,75 , 1))
xv = tf.convert_to_tensor(X_val.reshape(25313, 75, 1))
xt = tf.convert_to_tensor(X_test.reshape(41207, 75, 1))
yy = tf.convert_to_tensor(y_train.reshape(227811, 1))
yv = tf.convert_to_tensor(y_val.reshape(25313, 1))
yt = tf.convert_to_tensor(y_test.reshape(41207, 1))

In [13]:
yy= to_categorical(yy)
yv =  to_categorical(yv)
yt = to_categorical(yt)

In [14]:
model = keras.Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(75,1)))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))

model.add(Dropout(0.2))

model.add(MaxPooling1D(pool_size=4))

model.add(LSTM(32, activation = tf.keras.layers.LeakyReLU(alpha=0.1), dropout=0.2, recurrent_dropout = 0.2))

model.add(Flatten())

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(256, activation='relu'))

model.add(keras.layers.Dense(128, activation='relu'))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(6, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 73, 64)            256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 71, 64)            12352     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 69, 32)            6176      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 67, 32)            3104      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 65, 32)            3104      
_________________________________________________________________
dropout (Dropout)            (None, 65, 32)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 16, 32)            0

In [15]:
hist = model.fit(x = xx , y = yy, epochs=25, validation_data =(xv, yv), batch_size= 32)


Epoch 1/25
7120/7120 [==============================] - 513s 72ms/step - loss: 1.4774 - accuracy: 0.4403 - val_loss: 1.4352 - val_accuracy: 0.4481
Epoch 2/25
7120/7120 [==============================] - 518s 73ms/step - loss: 1.4145 - accuracy: 0.4542 - val_loss: 1.3897 - val_accuracy: 0.4553
Epoch 3/25
7120/7120 [==============================] - 518s 73ms/step - loss: 1.3891 - accuracy: 0.4605 - val_loss: 1.3762 - val_accuracy: 0.4643
Epoch 4/25
7120/7120 [==============================] - 406s 57ms/step - loss: 1.3673 - accuracy: 0.4678 - val_loss: 1.3591 - val_accuracy: 0.4692
Epoch 5/25
7120/7120 [==============================] - 377s 53ms/step - loss: 1.3504 - accuracy: 0.4740 - val_loss: 1.3438 - val_accuracy: 0.4753
Epoch 6/25
7120/7120 [==============================] - 379s 53ms/step - loss: 1.3341 - accuracy: 0.4800 - val_loss: 1.3181 - val_accuracy: 0.4812
Epoch 7/25
7120/7120 [==============================] - 376s 53ms/step - loss: 1.3214 - accuracy: 0.4850 - val_loss: 1

In [17]:
h = model.evaluate(xt,yt)

1288/1288 [==============================] - 7s 5ms/step - loss: 1.1902 - accuracy: 0.5391
